In [28]:
import numpy as np
import vplanet as vpl
import os
import sys
import pandas as pd
import copy
import matplotlib.pyplot as plt

sys.path.insert(0, os.path.join(os.pardir, os.pardir))

from parameter_sweep import Parameter_Sweep
from paths import path

In [3]:
seff_solar_2013 = np.array([1.7753, 1.0512, 1.0140, 0.3438, 0.3179, 0.2484])

# Row 1: Recent venus
# Row 2: Runaway greenhouse
# Row 3: Moist greenhouse
# Row 4: 
# Row 5: 2 AU limit
coeff_2013 = np.array([
    [1.4316e-4, 1.3242e-4, 8.1774e-5, 5.8942e-5, 5.4513e-5, 4.2588e-5], # Column 1 (a)
    [2.9875e-9, 1.5418e-8, 1.7063e-9, 1.6558e-9, 1.5313e-9, 1.1963e-9], # Column 2 (b)
    [-7.5702e-12, -7.9895e-12, -4.3241e-12, -3.0045e-12, -2.7786e-12, -2.1709e-12], # Column 3 (c)
    [-1.1635e-15, -1.8328E-15, -6.6462e-16, -5.2983e-16, -4.8997e-16, -3.8282e-16] # Column 4 (d)
])

# For 1 Earth mass.
# Row 1: Recent venus
# Row 2: Runaway greenhouse
# Row 3: Maximum greenhouse
# Row 4: Early mars
seff_solar_2014 = np.array([1.776, 1.107, 0.356, 0.32])

coeff_2014 = np.array([
    [2.136e-4, 1.332e-4, 6.171e-5, 5.547e-5], # Column 1 (a)
    [2.533e-8, 1.58e-8, 1.698e-9, 1.526e-9], # Column 2 (b)
    [-1.332e-11, -8.308e-12, -3.198e-12, -2.874e-12], # Column 3 (c)
    [-3.097e-15, -1.931e-15, -5.575e-16, -5.011e-16] # Column 4 (d)
])

# Solar luminosities, Kelvin.
def get_distance_2013(lum, teff):
    tstar = teff - 5780 # Kelvin.
    tvector = np.array([tstar, tstar**2, tstar**3, tstar**4])

    seff_2013 = seff_solar_2013 + np.matmul(tvector, coeff_2013)

    return np.sqrt(lum/seff_2013)

# Solar luminosities, Kelvin.
def get_distance_2014(lum, teff):
    tstar = teff - 5780 # Kelvin.
    tvector = np.array([tstar, tstar**2, tstar**3, tstar**4])

    seff_2014 = seff_solar_2014 + np.matmul(tvector, coeff_2014)

    return np.sqrt(lum/seff_2014)

In [22]:
def read_vplanet(filepath, output_options=None, usecols=None):
    if usecols == None:
        usecols = output_options

    if output_options == None:
        path_list = filepath.split(os.sep)

        directory = os.sep.join(path_list[:-1])

        body_file_name = path_list[-1].split('.')[-2] + '.in'

        with open(os.path.join(directory, body_file_name)) as body_file:
            body = body_file.read()

            output_options = body.split('saOutputOrder')[-1].replace('$', '').replace('\n', '').replace('-', '').strip()

            # Sequentially removes unneeded space.
            while output_options.find('  ') != -1:
                output_options = output_options.replace('  ', ' ')

        output_options = output_options.split(' ')

    return pd.read_csv(filepath, sep = ' ', names = output_options, index_col = False, usecols=usecols)


In [5]:
for d in os.listdir(os.curdir):
    # Skip the notebook file.
    if '.' in d or '_water' in d:
        continue

    p = os.path.join(d, 'star.star.forward')
    out = read_vplanet(p)

    mask = out['Age'] == 4.5e9

    temp = 5780 #temp = out['Temperature'][mask].iloc[0]
    lum = out['Luminosity'][mask].iloc[0]

    dist = get_distance_2014(lum, temp)

    output = 'HZ for {d} is: {a}-{b} AU'.format(d=d, a=dist[1], b=dist[2])

    print(output)

HZ for m0p8 is: 0.5102977503751515-0.8998548572102858 AU
HZ for m0p2 is: 0.06620435909745648-0.11674422248293947 AU
HZ for m1p1 is: 1.2868847141019824-2.2692819237455275 AU
HZ for m0p7 is: 0.36593793067361996-0.6452919381127107 AU
HZ for m1p2 is: 1.5499797475146282-2.733221542429373 AU
HZ for m0p3 is: 0.09966520287873788-0.17574879928306256 AU
HZ for m0p1 is: 0.028290658121484253-0.04988751391825234 AU
HZ for m0p5 is: 0.1816490755014541-0.32031848617316744 AU
HZ for m0p4 is: 0.13409673058156513-0.23646507212922305 AU
HZ for kv is: 0.6264682843462864-1.1047090216305424 AU
HZ for m0p9 is: 0.69953591533123-1.2335558813282643 AU
HZ for trappist is: 0.02269152152350059-0.04001404247893004 AU
HZ for m0p6 is: 0.2568079363450989-0.4528530034090855 AU
HZ for sun is: 0.9553042147547-1.6845755975379155 AU


In [32]:
parameter_sweep_dir = path('src', 'parameter_sweeps')

for d in os.listdir(parameter_sweep_dir):
    # Skip the notebook file.
    if '__' in d or '.' in d or 'synthesis' in d:
        continue

    for f in os.listdir(path(parameter_sweep_dir, d)):
        if '.forward' in f and not 'earth' in f and not 'water' in d:            
            out = read_vplanet(path(parameter_sweep_dir, d, f))

            mask = out['Age'] == 4.5e9

            temp = out['Temperature'][mask].iloc[0]
            lum = out['Luminosity'][mask].iloc[0]

            dist = get_distance_2014(lum, temp)

            central_hz = (dist[1] + dist[2]) / 2.0

            output = 'HZ for {d} is: {a}-{b} AU. Central HZ is {c}'.format(d=f, a=dist[1], b=dist[2], c=central_hz)

            print(output)

HZ for trappist.trappist_1.forward is: 0.024618766787709415-0.04975139624467579 AU. Central HZ is 0.037185081516192604
HZ for kv.kv.forward is: 0.64793159642507-1.164281274980237 AU. Central HZ is 0.9061064357026535
HZ for sol.sun.forward is: 0.9582115195960526-1.6920109431175816 AU. Central HZ is 1.325111231356817


In [5]:
os.listdir()

['m0p8',
 'm0p3_water',
 'm0p7_water',
 'get_hz_limits.ipynb',
 'm0p8_water',
 'm0p1_water',
 'm0p6_water',
 'm0p2',
 'sun_water',
 'm1p2_water',
 'm1p1',
 'm0p4_water',
 'm0p7',
 'm1p2',
 'm0p3',
 'm0p1',
 'm0p5',
 'run_sweep.py',
 'run_parameter_sweeps.ipynb',
 'm0p2_water',
 'm0p4',
 'kv',
 'm0p9',
 'm0p5_water',
 'm0p9_water',
 'trappist',
 'm0p6',
 'sun',
 'trappist_water',
 'kv_water',
 'm1p1_water']